In [1]:
import pandas as pd
import os
import datetime
import re
import json
import pymysql
from sqlalchemy import create_engine

In [2]:
## DATE GLOBAL VARS FOR FILENAMES
NOW = datetime.datetime.now()
TODAY = NOW.strftime("%Y.%m.%d")

## LOAD COLUMN MAPPING
with open('column_mapping.json', 'r') as f:
        COLMAP = json.load(f)

In [3]:
## LOAD DB CREDS AND OPEN CONNECTION
with open('db_creds.json', 'r') as f:
        db_creds = json.load(f)

usr = db_creds['username']
pwd = db_creds['password']
port = db_creds['port']
db = db_creds['database']
host = db_creds['host']

engine = create_engine('mysql+pymysql://{}:{}@{}:{}/{}?charset=utf8'.format(usr,pwd,host,port,db))

In [8]:
transaction =  '''
               START TRANSACTION;

                # CREATIVE META TEMP TABLE
                INSERT INTO creative_meta (id,creative_name,creative_type,creative_length)
                SELECT  id,
                        creative_name,
                        creative_type,
                        creative_length
                FROM    TEMP_creative as tmp
                ON      DUPLICATE KEY 
                UPDATE  creative_type = tmp.creative_type,
                        creative_length = tmp.creative_length;

                # PLACEMENTS TABLE
                INSERT INTO placements (campaign_id,id,placement_name,placement_start_date,placement_end_date,ad_type,package,package_id,site_dcm,booked_impressions)
                SELECT  campaign_id,id,placement_name,placement_start_date,placement_end_date,ad_type,package,package_id,site_dcm,booked_impressions
                FROM    TEMP_placement as tmp
                ON      DUPLICATE KEY 
                UPDATE  placement_start_date = tmp.placement_start_date,
                        placement_end_date = tmp.placement_end_date,
                        booked_impressions = tmp.booked_impressions,
                        package = tmp.package_id,
                        package_id = tmp.package_id,
                        booked_impressions = tmp.booked_impressions;

                # DCM TABLE
                INSERT INTO dcm (client_id,campaign_id,placement_id,creative_id,date_recorded,impressions,clicks,click_rate,total_conv,audio_comp,vid_prog,video_skip,video_plays,q1_completes,q2_completes,q3_completes,vid_completes,vid_view_rate,vid_views,video_avg_view_time
                )
                SELECT  client_id,
                        campaign_id,
                        placement_id,
                        creative_id,
                        date_recorded,
                        impressions,
                        clicks,
                        click_rate,
                        total_conv,
                        audio_comp,
                        vid_prog,
                        video_skip,
                        video_plays,
                        q1_completes,
                        q2_completes,
                        q3_completes,
                        vid_completes,
                        vid_view_rate,
                        vid_views,
                        video_avg_view_time
                FROM    TEMP_dcm as tmp
                ON      DUPLICATE KEY 
                UPDATE   
                        package_id = tmp.package_id,
                        impressions = tmp.impressions ;

                DROP TABLE TEMP_placement,TEMP_creative

                COMMIT;
                '''

In [9]:
def process_exports(d):
    log = []
    for export_type,export_path in d.items():
        print("Processing: {}".format(export_path))
        try:
            if export_type in ['dcm','creative','placement']:
                df = pd.read_csv(export_path,skiprows=12,skipfooter=1,engine='python')            

            elif export_type == 'double_verify':
                df = pd.read_csv(export_path)
                df['Date'] = pd.to_datetime(df['Date'])
                df.drop(labels=['Media Property', 'Placement Name'],axis=1,inplace=True)

            elif export_type == 'moat':
                df = pd.read_csv(export_path)      
        
            df.rename(columns=COLMAP[export_type],inplace=1)
            if df.shape[1] < 3:
                break
            df.to_csv('{}.csv'.format(export_type),index=False)

            df.to_sql('TEMP_{}'.format(export_type),con=engine,index=False,if_exists='fail')

        except Exception as e:
            print(e)
            log.append('Error for {} @ path: {}'.format(export_type,export_path))        
    return(log)

In [7]:
def main():
    os.chdir(os.path.expanduser('~/Downloads/exports'))
    d = {}
    for x in os.listdir():
        if bool(re.match(r"9003_DCM_Refresh_General",x)):
            d['dcm'] = x
        elif bool(re.match(r"9003_Creative_Meta_Refresh_General",x)):
            d['creative'] = x
        elif bool(re.match(r"9003_Placement_Overview_General",x)):
            d['placement'] = x
        elif bool(re.match(r"Moat Export",x)):
            d['moat'] = x
        elif bool(re.match(r"All Media Placement Daily Overview",x)):
            d['double_verify'] = x
        else:
            pass

    status = process_exports(d)
    #engine.excecute(transaction)
    print(status)

In [8]:
main()

Processing: 9003_Creative_Meta_Refresh_General_20180313_134811_695028141.csv
Processing: 9003_DCM_Refresh_General_20180314_095218_695500731.csv
Processing: 9003_Placement_Overview_General_20180313_134330_695014666.csv
['Success for creative @ path: 9003_Creative_Meta_Refresh_General_20180313_134811_695028141.csv', 'Success for dcm @ path: 9003_DCM_Refresh_General_20180314_095218_695500731.csv', 'Success for placement @ path: 9003_Placement_Overview_General_20180313_134330_695014666.csv']


## TO DO
- ~~write generalized processing function~~
- ~~JSON Dump/Load~~
- ~~SQL Connector~~
- Excecute cross table updates
    - DCM Tables
    - Moat tables
    - DV Tables
- Delete Temp Tables


In [9]:
result = engine.execute("select * from dcm LIMIT 2")

In [10]:
for x in result:
    print(x)

(4491231, 20685898, 213866168, 97636501, datetime.date(2018, 2, 9), 1, 1, 100.0, 3, 0, 0, 0, 1, 1, 0, 0, 0, 0.0, 0, 0.0)
(4491231, 20685898, 213866168, 97692297, datetime.date(2018, 2, 9), 1, 1, 100.0, 3, 0, 0, 0, 1, 1, 1, 1, 1, 0.0, 0, 0.0)
